# Using HTML and JavaScript with .NET Interactive

Creating visualizations for data is one of the key features of notebooks. In both Visual Studio Code and Jupyter, the frontend can render HTML, and there are numerous libraries available for .NET and Python the help create plots and visualizations. It's common in such libraries that you can write code that executes in the kernel, and the library handles the transformation into HTML and JavaScript for you. With .NET Interactive, there are some APIs available to simplify the process of directly writing your HTML and JavaScript. This enables you to create custom visualizations and directly access the broader ecosysytem of JavaScript libraries without needing wrapper libraries. You may choose to use these APIs directly or use them to create custom extensions to enrich the visualization of .NET types.

In this section, we'll take a look at:

* Emitting HTML
* Emitting JavaScript
* Accessing kernel data from client-side JavaScript code

## Emitting HTML

The simplest way to write some HTML to the client in .NET Interactive is to use the `#!html` magic command:

In [21]:
#!html
<b style="color:green">HTML!</b>

HTML!

Another way to write out HTML is to display or return a value of type `IHtmlContent`, which is used to signal that a `string` should not be HTML-encoded but rather treated as valid HTML:

In [22]:
#!csharp
using Microsoft.AspNetCore.Html;
new HtmlString("<b style=\"color:blue\">HTML!</b>")

HTML!

The `HTML` helper method is available for wrapping a string into an `IHtmlContent` instance, which will accomplish the same thing:

In [23]:
HTML("<b style=\"color:turquoise\">HTML!</b>")

HTML!

A fourth approach, if you'd like to avoid thinking about string escaping and HTML encoding, and you're writing your code in C#, is to use the `PocketView` API:

In [24]:
using static Microsoft.DotNet.Interactive.Formatting.PocketViewTags;
b[style: "color:orange"]("HTML!")

HTML!

`PocketView` is a C# domain-specific language for writing HTML. You can learn more about it [here](pocketview.md).

## Emitting JavaScript

Just like you can directly write HTML using a magic command, you can also scripts that will be run on the frontend. The simplest approach is again a magic command, either `#!javascript` or `#!js`:

In [25]:
#!html

<b id="target">HTML!</b>

HTML!

In [36]:
#!js

let e = document.getElementById('target');
e.style.color = 'magenta'

return e.outerHTML;

## Accessing kernel data from client-side JavaScript code

Most of the interesting work in your notebook is probably happening in the kernel, not in the client, so .NET Interactive gives you a way to access the data from the kernel. Any top-level variables declared in a kernel can be accessed from JavaScript running in the client using the `#!share` [magic command](magic-commands.md). 

Here's an example:

In [27]:
#!csharp

var x = "A value in the C# kernel";

In [28]:
#!html

<b id="targetx">HTML!</b>

HTML!

In [35]:
#!js
#!share x --from csharp

let e = document.getElementById('targetx');
e.innerText = x;

return e.outerHTML;

## Loading external JavaScript modules at runtime

Sometimes you might need to import JavaScript modules into your notebook. You can use the `interactive` object to do so.

Here is an example that configures RequireJS to load [D3.js](https://d3js.org/) from a CDN. `configureRequire` returns a function that can be used to load the module.

In [30]:
#!js
dtreeRequire = interactive.configureRequire({
    paths: {
        d3: "https://d3js.org/d3.v5.min"
    }
});

You can require the module by invoking `dtreeRequire`. Then, you can use `#!html` to inject an `svg` element and then `#!js` to load and call the `d3.v5.min.js` library.

In [31]:
#!html
<svg id="renderTarget" width=300 height=151></svg>

In [34]:
#!js
dtreeRequire(["d3"], d3 => {
    d3.select("svg#renderTarget")
    .append('circle')
    .attr('cx', 100)
    .attr('cy', 100)
    .attr('r', 50)
    .attr('stroke', 'black')
    .attr('fill', '#69a3b2');
});

The `interactive.configureRequire` is equivalent to [`require.config`](https://www.tutorialspoint.com/requirejs/requirejs_quick_guide.htm).

Doing this in .NET Interactive:

In [33]:
#!js
dtreeRequire = interactive.configureRequire({
    paths: {
        d3: "https://d3js.org/d3.v5.min"
    }
});

dtreeRequire(["d3"], d3 => { 
    console.log(d3);
});

is equivalent to doing this with RequireJS:

```js
requirejs.config({
    paths: {
        d3: "https://d3js.org/d3.v5.min"
    }
});

require(["d3"], d3 => { 
    console.log(d3);
});
```